## Setup

In [1]:
import os
import sys
from pathlib import Path
import random
import numpy as np
import pandas as pd
import tensorflow as tf

from pesuelogit.networks import build_tntp_network
from pesuelogit.etl import get_design_tensor, add_period_id

In [2]:
# Path management
main_dir = str(Path(os.path.abspath('')).parents[1])
os.chdir(main_dir)
print('main dir:', main_dir)

sys.path.append(os.path.join(main_dir, 'src'))

main dir: /Users/pablo/github/mate


In [3]:
%load_ext autoreload
%autoreload 2

# Internal modules
from mate.models import UtilityParameters, create_suelogit, compute_generation_factors
from mate.utils import load_k_shortest_paths
from mate.experiments import simulate_features, simulate_mate_data
from mate.etl import read_tntp_od

In [4]:
# Seed for reproducibility
_SEED = 2024
np.random.seed(_SEED)
random.seed(_SEED)
tf.random.set_seed(_SEED)

## Build network

In [5]:
network_name = 'SiouxFalls'
network = build_tntp_network(network_name=network_name, folderpath=os.getcwd() + "/input/tntp/")

## Read and load OD matrix

In [6]:
# Load true demand matrix
Q = read_tntp_od(network_name=network_name, folderpath=os.getcwd() + "/input/tntp/")
network.load_OD(Q=Q)

# Dense representation of O-D matrix by period
q_true = tf.stack([network.OD.q_true.flatten(), 0.8*network.OD.q_true.flatten()], axis = 0)
# Note: # Data from period 2 is generated with OD matrix that is 0.8 times the true OD for that period

Reading Q from external file
Matrix Q (24, 24) read in 0.1[s]                        

360600.0 trips were loaded among 528 o-d pairs


## Generate paths

In [7]:
load_k_shortest_paths(network=network, k=3, update_incidence_matrices=True)

1584 paths were loaded and incidence matrices were built


## Utility function

In [8]:
#Exogenous features of the utility function
_FEATURES_Z = ['tt_sd', 's']

utility_function = UtilityParameters(features_Y=['tt'],
                                     features_Z=_FEATURES_Z,
                                     true_values={'tt': -1, 'tt_sd': -1.3, 's': -3},
                                     initial_values={'tt': -1, 'tt_sd': -1.3, 's': -3},
                                     dtype = tf.float32
                                     )

## Simulation

In [9]:

# Generate node data
node_data = pd.DataFrame({'key': [node.key for node in network.nodes],
                          'income': np.random.rand(len(network.nodes)),
                          'population': np.random.rand(len(network.nodes))
                          })

In [10]:
# Generate link-specific data
n_observations = 300

df = simulate_features(links=network.links,
                       features_Z=_FEATURES_Z,
                       option='continuous',
                       time_variation=False,
                       range=(0, 1),
                       n_days = n_observations)

# Note: The value of the exogenous attributes varies between links but not between days

In [11]:
df['period'] = 1
df.loc[df.timepoint >= int(2/3*n_observations), 'period'] = 2

df = add_period_id(df, period_feature='period')

period_keys = df[['period', 'period_id']].drop_duplicates().reset_index().drop('index', axis=1).sort_values('period')
print(period_keys)

   period  period_id
0       1          0
1       2          1


In [12]:
X = get_design_tensor(Z=df[_FEATURES_Z + ['period_id']], n_links=len(network.links), n_timepoints=len(df['timepoint'].unique()))

n_periods = len(np.unique(X[:, :, -1].numpy().flatten()))

In [13]:
model = create_suelogit(network = network,
                        n_periods = n_periods,
                        reference_q = q_true,
                        features_Z = _FEATURES_Z,
                        utility_parameters = utility_function)

In [14]:
# Generation of spatio-temporal data in multiple periods

Y = simulate_mate_data(
    model = model,
    X = X,
    batch_size=1,
    # loss_metric=mse,
    max_epochs=200,
    optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=5e-3),
    threshold_relative_gap=1e-4,
    sd_x = 0.05,
    sd_t = 0.05,
    # coverage = 0.75
)


Computing gradient based equilibrium

Model training

Link flows and travel times were pretrained with single pass of traffic assignment

hyperparameters loss function: {'equilibrium': 1}

number of periods: 2, batch size: 1, threshold relative gap: 0.0001
training set -> timepoints: 300, obs [t x]: nan, coverage [t x]: nan

0/200: train mse=9.7e+07, avg theta = [-1.  -1.3 -3. ], avg rr = 1.30, avg theta fixed effect = 0, loss prop od=0, total trips=[3.61e+05 2.88e+05], avg alpha=0.15, avg beta=4, lambda eq=1, relative gap=0.71, train equilibrium loss=9.7e+07, time: 0.1

1/200: train mse=9.1e+07, avg theta = [-1.  -1.3 -3. ], avg rr = 1.30, avg theta fixed effect = 0, loss prop od=0, total trips=[3.61e+05 2.88e+05], avg alpha=0.15, avg beta=4, lambda eq=1, relative gap=0.68, train equilibrium loss=9.1e+07, time: 3.9

2/200: train mse=8.5e+07, avg theta = [-1.  -1.3 -3. ], avg rr = 1.30, avg theta fixed effect = 0, loss prop od=0, total trips=[3.61e+05 2.88e+05], avg alpha=0.15, avg be

In [15]:
# Generate a pandas dataframe
df['traveltime'], df['counts'] = tf.unstack(Y.numpy().reshape(-1, 2),axis = 1)

df = df.drop('period_id', axis = 1)

In [16]:
# Check if generation factors that are computed with observed link flows match the ground truth
generation_factors = compute_generation_factors(period_column=X[:, :, -1, None].numpy(),
                                                flow_column=Y[:, :, 1, None].numpy(), reference_period=0)
print(generation_factors)

period_id
0.0    1.000000
1.0    0.800562
Name: flow, dtype: float64


In [17]:
# Write file
output_file = network.key + '-link-data.csv'
output_dir = Path('output/network-data/' + network.key + '/links')
output_dir.mkdir(parents=True, exist_ok=True)

df.to_csv(output_dir / output_file, index=False)